In [1]:
import numpy as np  # this module handles arrays, but here we need it for its NaN value
import pandas as pd # this module contains a lot of tools for handling tabular data
import re

In [2]:
# define paths to the source files and eventual output file
#pathBottle='/ocean/eolson/MEOPAR/obs/Nina/All 2018 SoG bottle.xlsx'
#pathPhyto='/ocean/eolson/MEOPAR/obs/Nina/2015-2018 Abs phyto groups.xlsx'
pathBottle='/ocean/eolson/MEOPAR/obs/NemcekHPLC/All 2019 SoG bottle_revWithNew044.xlsx'
pathPhyto='/ocean/eolson/MEOPAR/obs/NemcekHPLC/2015-2019 SoG abs values New ALLO.xlsx'
pathOut='/ocean/eolson/MEOPAR/obs/NemcekHPLC/bottlePhytoMerged2019_NewALLO.csv'

In [3]:
#formatting function to get year-(3digit) format
def fmtCruise(istr):
    if re.fullmatch('[0-9]{4}-[0-9]{2}',istr): 
        sp=re.split('-',istr)
        rstr=sp[0]+'-0'+sp[1]
    elif re.fullmatch('[0-9]{4}-[0-9]{3}',istr):
        rstr=istr
    else:
        raise ValueError('Input had unexpected format:',istr)
    return rstr

In [4]:
# get names of sheets in notebook
with pd.ExcelFile(pathBottle,engine='openpyxl') as xl:
    sheets=xl.sheet_names
print(sheets)

['2019-001', '2019-007', '2019-016', '2019-044', '2019-005', '2019-045', '2019-050', '2019-023', '2019-062']


In [5]:
# load each sheet in the bottle Excel file and concatenate them together into one table
dfbotlist=list()
for sheet in sheets:
    df0=pd.read_excel(pathBottle,sheet_name=sheet,verbose=True,engine='openpyxl',
                      na_values=(-99,-99.9)) # read each sheet; include additional na values
    df0['Cruise']=fmtCruise(sheet)  # create and populate Cruise column based on sheet name
    dfbotlist.append(df0) # append the sheet to a list
dfbot=pd.concat(dfbotlist,ignore_index=True,sort=False) # concatenate the list into a single table

Reading sheet 2019-001
Reading sheet 2019-007
Reading sheet 2019-016


/home/eolson/anaconda3/envs/py39/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Reading sheet 2019-044
Reading sheet 2019-005
Reading sheet 2019-045
Reading sheet 2019-050
Reading sheet 2019-023
Reading sheet 2019-062


In [6]:
# Drop columns with no data in them
l1=set(dfbot.keys())
dfbot.dropna(axis=1,how='all',inplace=True)
print('removed empty columns:',l1-set(dfbot.keys()))

removed empty columns: {'Dimethylsulfoniopropionate_Total [nmol/L]', 'Flag:Dimethyl_Sulphide', 'Flag:Dimethylsulfoniopropionate_Total', 'Dimethyl_Sulphide [nmol/L]', 'Ammonium [umol/L]', 'Dimethylsulfoniopropionate_Dissolved [nmol/L]', 'Flag:Ammonium', 'Flag:Dimethylsulfoniopropionate_Dissolve'}


In [7]:
# list the column names in the resulting table
for el in sorted(dfbot.keys()):
    if not el.startswith('ADM') and not el.startswith('Flag') and not el.startswith('LOC') \
     and not el.startswith('Bottle'):
        print(el)

Chlorophyll:Extracted
Chlorophyll:Extracted [mg/m^3]
Comments by sample_numbeR
Conductivity:Primary [S/m]
Conductivity:Secondary
Conductivity:Secondary [S/m]
Cruise
Depth
Depth [metres]
Depth:Nominal [metres]
FIL:DATA DESCRIPTION
FIL:START TIME YYYY/MM/DD HH:MM:SS
File Name
Fluorescence:URU:Seapoint
Fluorescence:URU:Seapoint [mg/m^3]
Fluorescence:URU:Wetlabs [mg/m^3]
Nitrate_plus_Nitrite
Nitrate_plus_Nitrite [umol/L]
Number_of_bin_records
Oxygen:Dissolved
Oxygen:Dissolved [mL/L]
Oxygen:Dissolved [umol/kg]
Oxygen:Dissolved.1
Oxygen:Dissolved:SBE
Oxygen:Dissolved:SBE [mL/L]
Oxygen:Dissolved:SBE [umol/kg]
Oxygen:Dissolved:SBE.1
PAR
PAR [uE/m^2/sec]
PAR:Reference
PAR:Reference [uE/m^2/sec]
Phaeo-Pigment:Extracted
Phaeo-Pigment:Extracted [mg/m^3]
Phosphate
Phosphate [umol/L]
Pressure
Pressure [decibar]
Salinity:Bottle
Salinity:Bottle [PSS-78]
Salinity:T0:C0 [PSS-78]
Salinity:T1:C1
Salinity:T1:C1 [PSS-78]
Sample_Number
Silicate
Silicate [umol/L]
Temperature:Draw
Temperature:Draw [deg C (ITS9

In [8]:
for el in dfbot.keys():
    if 'Oxy' in el:
        print(el,np.max(dfbot[el]))

Oxygen:Dissolved:SBE [mL/L] 275.2
Oxygen:Dissolved:SBE [umol/kg] 447.5
Oxygen:Dissolved [mL/L] 9.571
Oxygen:Dissolved [umol/kg] 419.6
Flag:Oxygen:Dissolved 46.0
Oxygen:Dissolved:SBE 9.19
Oxygen:Dissolved:SBE.1 9.19
Oxygen:Dissolved 8.924
Oxygen:Dissolved.1 8.924


In [9]:
# no rows returned, so there are no rows with both primary and secondary temperature values
print(np.sum(dfbot['Depth [metres]']>=0),
      np.sum(dfbot['Pressure [decibar]']>=0),
      np.sum(dfbot['Pressure']>=0),
     np.sum(dfbot['Depth']>=0))
len(dfbot.loc[~(dfbot['Depth [metres]']>=0)&\
          ~(dfbot['Pressure [decibar]']>=0)&\
          (dfbot['Pressure']>=0)])

1104 1104 340 340


340

In [10]:
# no rows returned, so there are no rows with both primary and secondary temperature values
print(np.sum(dfbot['Temperature:Primary [deg C (ITS90)]']>=0),
      np.sum(dfbot['Temperature:Secondary [deg C (ITS90)]']>=0),
      np.sum(dfbot['Temperature:Secondary']>=0))
dfbot.loc[(np.array([int(ii) for ii in (dfbot['Temperature:Primary [deg C (ITS90)]']>=0)])+\
          np.array([int(ii) for ii in (dfbot['Temperature:Secondary [deg C (ITS90)]']>=0)])+\
           np.array([int(ii) for ii in (dfbot['Temperature:Secondary']>=0)]))>1]

726 378 340


,Zone,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,ADM:MISSION,LOC:STATION,ADM:PROJECT,YYYY/MM/DD HH:MM:SS,...,Temperature:Draw,Salinity:Bottle,Chlorophyll:Extracted,Phaeo-Pigment:Extracted,Oxygen:Dissolved,Oxygen:Dissolved.1,Nitrate_plus_Nitrite,Silicate,Phosphate,Transmissivity:Green [*/metre]


In [11]:
# no rows returned, so there are no rows with both primary and secondary temperature values
print(np.sum(dfbot['Fluorescence:URU:Seapoint [mg/m^3]']>=0),
      np.sum(dfbot['Fluorescence:URU:Seapoint']>=0),
      np.sum(dfbot['Fluorescence:URU:Wetlabs [mg/m^3]']>=0))
dfbot.loc[(np.array([int(ii) for ii in (dfbot['Fluorescence:URU:Seapoint [mg/m^3]']>=0)])+\
          np.array([int(ii) for ii in (dfbot['Fluorescence:URU:Seapoint']>=0)])+\
           np.array([int(ii) for ii in (dfbot['Fluorescence:URU:Wetlabs [mg/m^3]']>=0)]))>1]

993 340 110


,Zone,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,ADM:MISSION,LOC:STATION,ADM:PROJECT,YYYY/MM/DD HH:MM:SS,...,Temperature:Draw,Salinity:Bottle,Chlorophyll:Extracted,Phaeo-Pigment:Extracted,Oxygen:Dissolved,Oxygen:Dissolved.1,Nitrate_plus_Nitrite,Silicate,Phosphate,Transmissivity:Green [*/metre]


In [12]:
# no rows returned, so there are no rows with both both salinity fields
print(np.sum(dfbot['Salinity:T0:C0 [PSS-78]']>=0),
      np.sum(dfbot['Salinity:T1:C1 [PSS-78]']>=0),
      np.sum(dfbot['Salinity:T1:C1']>=0))
dfbot.loc[(np.array([int(ii) for ii in (dfbot['Salinity:T0:C0 [PSS-78]']>=0)])+\
           np.array([int(ii) for ii in (dfbot['Salinity:T1:C1 [PSS-78]']>=0)])+\
           np.array([int(ii) for ii in (dfbot['Salinity:T1:C1']>=0)]))>1]

726 378 340


,Zone,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,ADM:MISSION,LOC:STATION,ADM:PROJECT,YYYY/MM/DD HH:MM:SS,...,Temperature:Draw,Salinity:Bottle,Chlorophyll:Extracted,Phaeo-Pigment:Extracted,Oxygen:Dissolved,Oxygen:Dissolved.1,Nitrate_plus_Nitrite,Silicate,Phosphate,Transmissivity:Green [*/metre]


In [13]:
# no rows returned, so there are no rows with both primary and secondary temperature values
print(np.sum(dfbot['Pressure [decibar]']>=0),
      np.sum(dfbot['Pressure']>=0))
dfbot.loc[(dfbot['Pressure [decibar]']>=0)&\
          (dfbot['Pressure']>=0)]

1104 340


,Zone,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,ADM:MISSION,LOC:STATION,ADM:PROJECT,YYYY/MM/DD HH:MM:SS,...,Temperature:Draw,Salinity:Bottle,Chlorophyll:Extracted,Phaeo-Pigment:Extracted,Oxygen:Dissolved,Oxygen:Dissolved.1,Nitrate_plus_Nitrite,Silicate,Phosphate,Transmissivity:Green [*/metre]


In [14]:
def subval(idf,colList):
    # first value in colList should be the column you are going to keep
    # follow with other columns that will be used to fill in when that column is NaN
    # in order of precedence
    if len(colList)==2:
        idf[colList[0]]=[r[colList[0]] if not pd.isna(r[colList[0]]) \
                         else  r[colList[1]] for i,r in idf.iterrows()]
    elif len(colList)==3:
        idf[colList[0]]=[r[colList[0]] if not pd.isna(r[colList[0]]) \
                         else  r[colList[1]] if not pd.isna(r[colList[1]]) \
                         else r[colList[2]] for i,r in idf.iterrows()]
    else:
        raise NotImplementedError('Add to code to handle this case')
    idf.drop(columns=list(colList[1:]),inplace=True)
    return idf

In [15]:
# there are some duplicate columns here; handle them:
dfbot=subval(dfbot,('Depth [metres]','Depth'))
dfbot=subval(dfbot,('Pressure [decibar]','Pressure'))
dfbot=subval(dfbot,('Temperature:Primary [deg C (ITS90)]',
                    'Temperature:Secondary [deg C (ITS90)]',
                    'Temperature:Secondary'))
dfbot=subval(dfbot,('Salinity:T0:C0 [PSS-78]',
                    'Salinity:T1:C1 [PSS-78]',
                    'Salinity:T1:C1'))
dfbot=subval(dfbot,('Fluorescence:URU:Seapoint [mg/m^3]',
                    'Fluorescence:URU:Wetlabs [mg/m^3]',
                    'Fluorescence:URU:Seapoint'))
dfbot=subval(dfbot,('PAR [uE/m^2/sec]','PAR'))
dfbot=subval(dfbot,('PAR:Reference [uE/m^2/sec]','PAR:Reference'))
dfbot=subval(dfbot,('Oxygen:Dissolved:SBE [mL/L]',
                    'Oxygen:Dissolved:SBE',
                    'Oxygen:Dissolved:SBE.1'))
dfbot=subval(dfbot,('Oxygen:Dissolved [mL/L]', 'Oxygen:Dissolved', 'Oxygen:Dissolved.1'))
dfbot=subval(dfbot,('Transmissivity [*/metre]','Transmissivity2 [*/metre]','Transmissivity'))
dfbot=subval(dfbot,('Nitrate_plus_Nitrite [umol/L]', 'Nitrate_plus_Nitrite'))
dfbot=subval(dfbot,('Silicate [umol/L]','Silicate'))
dfbot=subval(dfbot,('Phosphate [umol/L]','Phosphate'))
dfbot=subval(dfbot,('Phaeo-Pigment:Extracted [mg/m^3]','Phaeo-Pigment:Extracted'))
dfbot=subval(dfbot,('Chlorophyll:Extracted [mg/m^3]','Chlorophyll:Extracted'))
dfbot=subval(dfbot,('Conductivity:Primary [S/m]',
                    'Conductivity:Secondary [S/m]',
                    'Conductivity:Secondary'))
dfbot=subval(dfbot,('FIL:START TIME YYYY/MM/DD HH:MM:SS','YYYY/MM/DD HH:MM:SS'))
dfbot=subval(dfbot,('Zone', 'Zone.1'))
dfbot=subval(dfbot,('Temperature:Draw [deg C (ITS90)]','Temperature:Draw'))
dfbot=subval(dfbot,('Salinity:Bottle [PSS-78]','Salinity:Bottle'))

dfbot.rename(columns={'Temperature:Primary [deg C (ITS90)]':'Temperature [deg C (ITS90)]'},
             inplace=True)
dfbot.rename(columns={'Salinity:T0:C0 [PSS-78]':'Salinity [PSS-78]'},
             inplace=True)
dfbot.rename(columns={'Fluorescence:URU:Seapoint [mg/m^3]':'Fluorescence:URU [mg/m^3]'},
             inplace=True)
dfbot.rename(columns={'Conductivity:Primary [S/m]':'Conductivity [S/m]'},
             inplace=True)

In [16]:
# define a function that will be applied to the values in the index column;
# this makes it easier to drop non-data rows later
def convertIndex(val):
    try:
        x =int(val)
    except ValueError:
        x=np.nan
    return x

In [17]:
# load the  phytoplankton data with the following options:
#   sheet_name='???? CHEMTAX abs results'  -> choose the  sheet
#   usecols='A:I,T:AC'   -> read only columns A:I and T:AC from the Excel sheet
#   skiprows=2     -> start reading at the 3rd row of the sheet, 
#                     which contains the column headings
#   converters={'Index': convertIndex,}   -> apply the function defined above to the Index column
#   verbose = True   -> print extra information/ warnings/ errors
dfPhyto=pd.read_excel(pathPhyto,sheet_name='2019',engine='openpyxl',
                      skiprows=0,converters={'Sample#': convertIndex,},
                      verbose=True)

Reading sheet 2019


In [18]:
# display rows 48 to 59 of the resulting table
dfPhyto[48:60]

,Cruise,Date,Station,Sample#,rep,depth,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA
48,2019-016,2019-04-11 00:00:00,102,318.0,mean,0,1.81861,0.254111,0.0210325,0.106842,0.0526873,0.222644,0,0.00369154,0.0117118,2.49133
49,2019-016,2019-04-11 00:00:00,75,332.0,mean,0,1.66103,0.145549,0.0161815,0.113534,0.0543728,0.236785,0,0.00986979,0.00749211,2.24481
50,2019-016,2019-04-11 00:00:00,72,345.0,mean,0,1.69253,0.0607043,0.0273842,0.20995,0.107062,0.235914,0.0171222,0.019343,0.00543582,2.37545
51,2019-016,2019-04-11 00:00:00,69,357.0,mean,0,1.20815,0.189921,0.0237731,0.219837,0.0697194,0.275599,0,0.0128893,0.00430002,2.00419
52,2019-016,2019-04-11 00:00:00,ADCP,367.0,mean,0,0.277147,0.0376473,0.0182011,0.222086,0.0207715,0.0508863,0.0027147,0.00533482,0.00312848,0.637917
53,2019-016,2019-04-11 00:00:00,65,377.0,mean,0,0.91255,0.0374169,0.0295607,0.226087,0.046938,0.232094,0.00658676,0.0116996,0.00512161,1.50806
54,2019-016,2019-04-11 00:00:00,63,380.0,mean,0,0.704281,0.0266105,0.0317602,0.254517,0.0525936,0.207993,0.00172955,0.0059513,0.00552183,1.29096
55,2019-016,2019-04-11 00:00:00,62,391.0,mean,0,0.52177,0.0800526,0.0118601,0.117773,0.0196437,0.0626985,0.0115089,0,0.0046034,0.82991
56,2019-016,2019-04-12 00:00:00,SC04,394.0,mean,0,3.5872,0.139061,0,0.112151,0.0294817,0.0468341,0,0,0.0152213,3.92995
57,Cruise,Date,Station,NaN,rep,depth,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA


In [19]:
# now, drop any rows from the table that have NaN values in either of the columns
#  'Index' or 'TchlA (ug/L)'
# This is why we applied a function to the Index column to make sure all 
#  non-numeric Index values would have a consistent NaN entry, making them easy to identify
#  and remove
dfPhyto.dropna(subset=['Sample#', 'TchlA'],how='any',inplace=True)

In [20]:
# pandas creates its own index, and after dropping rows I like to reset it -
# this is just for convenience
dfPhyto.reset_index(drop=True,inplace=True)

In [21]:
# apply formatting function all rows in Cruise column to get year-3digit format
dfPhyto['Cruise']=[fmtCruise(ii) for ii in dfPhyto['Cruise']]

In [22]:
# display part of the table, confirming that non-data rows have been removed
dfPhyto[48:60]

,Cruise,Date,Station,Sample#,rep,depth,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA
48,2019-016,2019-04-11 00:00:00,75,332.0,mean,0,1.66103,0.145549,0.0161815,0.113534,0.0543728,0.236785,0,0.00986979,0.00749211,2.24481
49,2019-016,2019-04-11 00:00:00,72,345.0,mean,0,1.69253,0.0607043,0.0273842,0.20995,0.107062,0.235914,0.0171222,0.019343,0.00543582,2.37545
50,2019-016,2019-04-11 00:00:00,69,357.0,mean,0,1.20815,0.189921,0.0237731,0.219837,0.0697194,0.275599,0,0.0128893,0.00430002,2.00419
51,2019-016,2019-04-11 00:00:00,ADCP,367.0,mean,0,0.277147,0.0376473,0.0182011,0.222086,0.0207715,0.0508863,0.0027147,0.00533482,0.00312848,0.637917
52,2019-016,2019-04-11 00:00:00,65,377.0,mean,0,0.91255,0.0374169,0.0295607,0.226087,0.046938,0.232094,0.00658676,0.0116996,0.00512161,1.50806
53,2019-016,2019-04-11 00:00:00,63,380.0,mean,0,0.704281,0.0266105,0.0317602,0.254517,0.0525936,0.207993,0.00172955,0.0059513,0.00552183,1.29096
54,2019-016,2019-04-11 00:00:00,62,391.0,mean,0,0.52177,0.0800526,0.0118601,0.117773,0.0196437,0.0626985,0.0115089,0,0.0046034,0.82991
55,2019-016,2019-04-12 00:00:00,SC04,394.0,mean,0,3.5872,0.139061,0,0.112151,0.0294817,0.0468341,0,0,0.0152213,3.92995
56,2019-044,2019-04-29 00:00:00,GEO1,9.0,mean,0,5.62557,0.409893,0.3874,0.241505,0.0396982,0.155595,0.070028,0.117273,0.0390135,7.08597
57,2019-044,2019-04-30 00:00:00,39,10.0,mean,0,8.91175,0.803637,0.238999,0.523477,0.23522,0.153088,0.120047,0.100953,0,11.0872


In [23]:
# due to repeated column names in the original spreadsheet, '.1' was appended to the names
# of the phytoplankton columns; 
# these lines correct the column names, removing the '.1':
renameDict=dict()
for colName in dfPhyto.keys():
    if colName.endswith('.1'):
        renameDict[colName]=colName.split('.1')[0]
dfPhyto.rename(columns=renameDict,inplace=True)

In [24]:
dfPhyto

,Cruise,Date,Station,Sample#,rep,depth,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA
0,2019-001,2019-02-20 00:00:00,14,729.0,mean,0,0.0331146,0.101655,0.123836,0.3117,0.00876211,0.1462,0.0225388,0.0234123,0.00090972,0.772129
1,2019-001,2019-02-20 00:00:00,12,732.0,mean,0,0.156416,0.17666,0.112519,0.237514,0.0127872,0.179982,0.0256941,0.0165342,0.000955474,0.919063
2,2019-001,2019-02-20 00:00:00,22,743.0,mean,0,0.356374,0.340304,0.125008,0.315779,0.0132582,0.07692,0.0203337,0.0362638,0.00278554,1.28703
3,2019-001,2019-02-20 00:00:00,CPF2,746.0,mean,0,0.0606656,0.116013,0.135416,0.332063,0.007642,0.159874,0.0260862,0.0274673,0.00137643,0.866603
4,2019-001,2019-02-20 00:00:00,CPF1,749.0,mean,0,0.404852,0.41367,0.131683,0.197787,0.0128683,0.0582133,0.0158865,0.0375323,0.00585506,1.27835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2019-062,2019-10-05 00:00:00,IND0,349.0,mean,0,9.2645,0.0111426,0.344424,0.291775,0.108978,0.109122,0.0927235,0.0746044,0,10.2973
172,2019-062,2019-10-05 00:00:00,IND2,361.0,mean,0,6.2459,0.18038,0.377089,0.237736,0.0688538,0.112986,0.0723897,0.0478686,0.0262837,7.36948
173,2019-062,2019-10-05 00:00:00,IND4,375.0,mean,0,8.57436,0,0.617925,0.224924,0.147793,0.254775,0.0411009,0.141533,0.0670786,10.0695
174,2019-062,2019-10-05 00:00:00,IND7,389.0,mean,0,7.17251,0,0.651271,4.73232,0.330853,0.0166483,0.0334197,0.152275,0.169401,13.2587


In [25]:
# This is the important step- join the two tables ('left' and 'right'), 
#  matching the cruise IDs and sample numbers
#   how='outer'  -> all rows from both the left and the right tables will be included, 
#                   even if they cannot be matched; this makes it easy to check for 
#                   unmatched data later
#   left_on  specifies the name of the column to match in the left table (dfbot) 
#   right_on specifies the name of the column to match in the right table (dfPhyto)
dfout = pd.merge(dfbot, dfPhyto,  how='outer', 
                 left_on=['Cruise','Sample_Number'], right_on = ['Cruise','Sample#'])

In [26]:
# Identify cases where phytoplankton data were matched to multiple samples in bottle data:
dftest=pd.merge(dfbot, dfPhyto,how='right', left_on=['Cruise','Sample_Number'],right_on = ['Cruise','Sample#'])


In [27]:
# show the column names in the resulting table
dfout.keys()

Index(['Zone', 'LOC:EVENT_NUMBER', 'LOC:LATITUDE', 'LOC:LONGITUDE',
       'LOC:WATER DEPTH', 'ADM:SCIENTIST', 'ADM:MISSION', 'LOC:STATION',
       'ADM:PROJECT', 'Bottle_Number', 'Bottle:Firing_Sequence',
       'Sample_Number', 'Pressure [decibar]', 'Depth [metres]',
       'Temperature [deg C (ITS90)]', 'Transmissivity [*/metre]',
       'Fluorescence:URU [mg/m^3]', 'PAR [uE/m^2/sec]', 'Salinity [PSS-78]',
       'Oxygen:Dissolved:SBE [mL/L]', 'Oxygen:Dissolved:SBE [umol/kg]',
       'Conductivity [S/m]', 'Number_of_bin_records',
       'Chlorophyll:Extracted [mg/m^3]', 'Flag:Chlorophyll:Extracted',
       'Phaeo-Pigment:Extracted [mg/m^3]', 'Nitrate_plus_Nitrite [umol/L]',
       'Flag:Nitrate_plus_Nitrite', 'Silicate [umol/L]', 'Flag:Silicate',
       'Phosphate [umol/L]', 'Flag:Phosphate', 'Comments by sample_numbeR',
       'Cruise', 'FIL:START TIME YYYY/MM/DD HH:MM:SS',
       'Depth:Nominal [metres]', 'Temperature:Draw [deg C (ITS90)]',
       'Salinity:Bottle [PSS-78]', 'Flag

### Checks

In [28]:
temp=dftest.groupby(['Cruise','Sample#']).agg({'Cruise':['count']})
temp.columns = ['icount']

In [29]:
np.unique(temp.icount)

array([1, 2])

In [30]:
# check for Phyto samples matched to multiple bottle samples:
temp.loc[temp.icount>1]

icount
Cruise   Sample#        
2019-062 349.0         2
         361.0         2
         375.0         2
         389.0         2
         395.0         2

In [31]:
# check for phyto samples not matched to bottle samples:
temp.loc[temp.icount==0]

,,icount
Cruise,Sample#,


In [32]:
temp2=dfout.groupby(['Cruise','Sample_Number']).agg({'Cruise':['count']})
temp2.columns = ['icount']
# this will catch phyto matched to multiple bottle but also bottle with duplicate sample numbers per cruise:
temp2.loc[temp2.icount>1]

icount
Cruise   Sample_Number        
2019-062 349                 2
         361                 2
         375                 2
         389                 2
         395                 2

In [33]:
# check for phyto samples not matched to bottle samples:
temp.loc[temp.icount==0]

,,icount
Cruise,Sample#,


In [34]:
# if the output table is longer than either of the input tables, some columns were not matched
len(dfout), len(dfPhyto), len(dfbot)

(1449, 176, 1444)

In [35]:
# Check that the number of cells with data in the 'Cyanobacteria' column is 
#  the same for the input and output tables to show that no rows are missing:
np.sum(dfPhyto['Cyanobacteria']>=0), np.sum(dfout['Cyanobacteria']>=0)

(176, 176)

In [36]:
# If there were data rows from the phytoplankton table that were not matched to 
#  rows from the bottle table, their indices from the phytoplankton table would be 
#  displayed below (the series [] would not be empty)
print(dfout.loc[dfout['ADM:SCIENTIST'].isna(),['Cruise','Sample#']])

Empty DataFrame
Columns: [Cruise, Sample#]
Index: []


In [37]:
dfout.loc[dfout['ADM:SCIENTIST'].isna(),['Sample#','Cruise','Diatoms-1','Prasinophytes']]

,Sample#,Cruise,Diatoms-1,Prasinophytes


In [38]:
# drop repetetive/unecessary columns:
dfout.drop(labels=['Date', 'Station', 'Sample#', 'rep',
                   'depth',],axis=1,inplace=True)

In [39]:
# truncate phyto group values to 3 decimal places:
for col in ('Cyanobacteria', 'Prasinophytes', 'Cryptophytes', 'Diatoms-1',
       'Diatoms-2', 'Dinoflagellates-1', 'Haptophytes', 'Dictyo', 'Raphido',
       'TchlA'):
    dfout[col]=[np.round(ii,decimals=3) for ii in dfout[col]] # use list comprehension to set values for entire column

In [40]:
# Check for unexpected depths
dfout.loc[((dfout['Pressure [decibar]']>10)|(dfout['Depth [metres]']>10))&(dfout['Diatoms-1']>=0)]#,
#         ['FIL:START TIME YYYY/MM/DD HH:MM:SS','Pressure [decibar]','Depth [metres]','Diatoms-1']]

,Zone,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,ADM:MISSION,LOC:STATION,ADM:PROJECT,Bottle_Number,...,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA


In [41]:
dfout['Cyanobacteria']

0         NaN
1         NaN
2       0.001
3         NaN
4         NaN
        ...  
1444      NaN
1445      NaN
1446      NaN
1447    0.002
1448    0.009
Name: Cyanobacteria, Length: 1449, dtype: float64

In [42]:
# now write the output table to a .csv file:
dfout.to_csv(pathOut, index=False)  

In [43]:
dfout.keys()

Index(['Zone', 'LOC:EVENT_NUMBER', 'LOC:LATITUDE', 'LOC:LONGITUDE',
       'LOC:WATER DEPTH', 'ADM:SCIENTIST', 'ADM:MISSION', 'LOC:STATION',
       'ADM:PROJECT', 'Bottle_Number', 'Bottle:Firing_Sequence',
       'Sample_Number', 'Pressure [decibar]', 'Depth [metres]',
       'Temperature [deg C (ITS90)]', 'Transmissivity [*/metre]',
       'Fluorescence:URU [mg/m^3]', 'PAR [uE/m^2/sec]', 'Salinity [PSS-78]',
       'Oxygen:Dissolved:SBE [mL/L]', 'Oxygen:Dissolved:SBE [umol/kg]',
       'Conductivity [S/m]', 'Number_of_bin_records',
       'Chlorophyll:Extracted [mg/m^3]', 'Flag:Chlorophyll:Extracted',
       'Phaeo-Pigment:Extracted [mg/m^3]', 'Nitrate_plus_Nitrite [umol/L]',
       'Flag:Nitrate_plus_Nitrite', 'Silicate [umol/L]', 'Flag:Silicate',
       'Phosphate [umol/L]', 'Flag:Phosphate', 'Comments by sample_numbeR',
       'Cruise', 'FIL:START TIME YYYY/MM/DD HH:MM:SS',
       'Depth:Nominal [metres]', 'Temperature:Draw [deg C (ITS90)]',
       'Salinity:Bottle [PSS-78]', 'Flag